<a href="https://colab.research.google.com/github/BenjaminScialom/Finance/blob/master/Financial_web_scraping.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

### **Motivation**

Getting financial statment data online is quite difficult. Here, I aim at automating this process of getting finanical data from YahooFinance!

### **Libraries**

I will use 4 libraries: 


1.   Pandas : To deal with the large amount of data
2.   urllib.request: to manage the webscrapping part. it basically get all the information on a specific web page.
3.   BeautifulSoup: To take off the web language around the data I need.
4.   Datetime: To get the current year automatically










In [1]:
import pandas as pd
from bs4 import BeautifulSoup
import urllib.request as ur
from datetime import date

### **Build URL**

For instance: https://finance.yahoo.com/quote/MSFT/financials?p=AAPL

In [2]:
ticker = 'MSFT'
# URL link 

url_IncomeStmt = 'https://finance.yahoo.com/quote/%s/financials?p=%s' %(ticker,ticker)
url_BalanceSheet = 'https://finance.yahoo.com/quote/%s/balance-sheet?p=%s' %(ticker,ticker)
url_CashFlowStmt = 'https://finance.yahoo.com/quote/%s/cash-flow?p=%s' %(ticker,ticker)

## **Get  Fundamentals data:**
Enter ticker to get Income statement, Balance sheet and cash flow statement

In [24]:
def get_Income_statement(ticker):
  url_IncomeStmt = 'https://finance.yahoo.com/quote/%s/financials?p=%s' %(ticker,ticker)
  read_data_IS = ur.urlopen(url_IncomeStmt).read() 
  raw_IS = BeautifulSoup(read_data_IS,'lxml')

  ls= [] # Create empty list
  for data in raw_IS.find_all('div'): 
    #Find all data structure that is 'div'
    ls.append(data.string) # add each element one by one to the list

  CleanData_IS = list(filter(None,ls))
  # fixing some issues
  CleanData_IS= CleanData_IS[13:]
  CleanData_IS.remove('Quarterly')
  CleanData_IS.remove('Expand All')
  CleanData_IS.insert(6,'Total Revenue')
  CleanData_IS.insert(24,'Operating Expense')
  CleanData_IS.insert(36,'Net Non-Operating Interest Income Expense')
  CleanData_IS.insert(42,'Other Income Expense')
  CleanData_IS.insert(60,'Net Income Common Stockholders ')
  data_IS = list(zip(*[iter(CleanData_IS)]*6))

  # Build Dataframe
  Income_stmt = pd.DataFrame(data_IS[0:])
  Income_stmt.iloc[0,0] =""
  Income_stmt.columns = Income_stmt.iloc[0] # Name columns to first row of dataframe
  Income_stmt = Income_stmt.iloc[1:,] # start to read 1st row
  Income_stmt = Income_stmt.T # transpose dataframe
  Income_stmt.columns = Income_stmt.iloc[0] #Name columns to first row of dataframe
  Income_stmt.drop(Income_stmt.index[0],inplace=True) #Drop first index row
  Income_stmt.index.name = '' # Remove the index name
  Income_stmt.rename(index={'ttm': '%s/%s/%s' %(date.today().month,date.today().day,date.today().year)},inplace=True) #Rename ttm in index columns to end of the year
  Income_stmt.index = pd.to_datetime(Income_stmt.index.values,infer_datetime_format=True) # Converting the index as date

  return Income_stmt

In [25]:
IS = get_Income_statement(ticker='AAPL')

In [26]:
IS

,Total Revenue,Cost of Revenue,Gross Profit,Operating Expense,Operating Income,Net Non-Operating Interest Income Expense,Other Income Expense,Pretax Income,Tax Provision,Net Income Common Stockholders,Diluted NI Available to Com Stockholders,Basic EPS,Diluted EPS,Basic Average Shares,Diluted Average Shares,Total Operating Income as Reported,Total Expenses,Net Income from Continuing & Discontinued Operation,Normalized Income,Interest Income,Interest Expense,Net Interest Income,EBIT,EBITDA,Reconciled Cost of Revenue,Reconciled Depreciation,Net Income from Continuing Operation Net Minority Interest,Normalized EBITDA,Tax Rate for Calcs,Tax Effect of Unusual Items
2020-11-17,"273,857,000","169,277,000","104,580,000","37,442,000","67,138,000","1,052,000","127,000","68,317,000","9,893,000","58,424,000","58,424,000",-,-,-,-,"67,138,000","206,719,000","58,424,000","58,424,000","4,101,000","3,049,000","1,052,000","71,366,000","82,899,000","169,277,000","11,533,000","58,424,000","82,899,000",0,0
2020-09-30,"274,515,000","169,559,000","104,956,000","38,668,000","66,288,000","890,000","-87,000","67,091,000","9,680,000","57,411,000","57,411,000",0.0033,0.0033,"17,352,119","17,528,214","66,288,000","208,227,000","57,411,000","57,411,000","3,763,000","2,873,000","890,000","69,964,000",-,"169,559,000","11,056,000","57,411,000","81,020,000",0,0
2019-09-30,"260,174,000","161,782,000","98,392,000","34,462,000","63,930,000","1,385,000","422,000","65,737,000","10,481,000","55,256,000","55,256,000",0.003,0.003,"18,471,336","18,595,652","63,930,000","196,244,000","55,256,000","55,256,000","4,961,000","3,576,000","1,385,000","69,313,000",-,"161,782,000","12,547,000","55,256,000","81,860,000",0,0
2018-09-30,"265,595,000","163,756,000","101,839,000","30,941,000","70,898,000","2,446,000","-441,000","72,903,000","13,372,000","59,531,000","59,531,000",0.003,0.003,"19,821,508","20,000,436","70,898,000","194,697,000","59,531,000","59,531,000","5,686,000","3,240,000","2,446,000","76,143,000",-,"163,756,000","10,903,000","59,531,000","87,046,000",0,0
2017-09-30,"229,234,000","141,048,000","88,186,000","26,842,000","61,344,000","2,878,000","-133,000","64,089,000","15,738,000","48,351,000","48,351,000",0.0023,0.0023,"20,868,968","21,006,768","61,344,000","167,890,000","48,351,000","48,351,000","5,201,000","2,323,000","2,878,000","66,412,000",-,"141,048,000","10,157,000","48,351,000","76,569,000",0,0


In [27]:
def get_balance_sheet(ticker):
  
  url_BalanceSheet = 'https://finance.yahoo.com/quote/%s/balance-sheet?p=%s' %(ticker,ticker)
  read_data_BS = ur.urlopen(url_BalanceSheet).read() 
  raw_BS = BeautifulSoup(read_data_BS,'lxml')

  ls_BS= [] 
  for data in raw_BS.find_all('div'): 
    ls_BS.append(data.string) 

  CleanData_BS = list(filter(None,ls_BS))
  CleanData_BS = CleanData_BS[10:]
  CleanData_BS.remove('Quarterly')
  CleanData_BS.remove('Income Statement')
  CleanData_BS.remove('Expand All')
  CleanData_BS.remove('Cash Flow')
  CleanData_BS.remove('Balance Sheet')
  CleanData_BS.insert(5,'Total Assets')
  CleanData_BS.insert(10,'Total Liabilities')
  CleanData_BS.insert(15,'Total Equity')

  data_BS = list(zip(*[iter(CleanData_BS)]*5))

  balance_sheet = pd.DataFrame(data_BS[:-1])
  balance_sheet.iloc[0,0] =""
  balance_sheet.columns = balance_sheet.iloc[0] # Name columns to first row of dataframe
  balance_sheet = balance_sheet.iloc[1:,] # start to read 1st row
  balance_sheet = balance_sheet.T # transpose dataframe
  balance_sheet.columns = balance_sheet.iloc[0] #Name columns to first row of dataframe
  balance_sheet.drop(balance_sheet.index[0],inplace=True) #Drop first index row
  balance_sheet.index.name = '' # Remove the index name
  balance_sheet.rename(index={'ttm': '%s/%s/%s' %(date.today().month,date.today().day,date.today().year)},inplace=True) #Rename ttm in index columns to end of the year
  balance_sheet.index = pd.to_datetime(balance_sheet.index.values,infer_datetime_format=True) # Converting the index as date

  return balance_sheet

In [28]:
BS = get_balance_sheet(ticker='AAPL')

In [29]:
BS

,Total Assets,Total Liabilities,Total Equity,Total Capitalization,Common Stock Equity,Net Tangible Assets,Working Capital,Invested Capital,Tangible Book Value,Total Debt,Net Debt,Share Issued,Ordinary Shares Number
2020-09-30,"323,888,000","258,549,000","65,339,000","164,006,000","65,339,000","65,339,000","38,321,000","177,775,000","65,339,000","112,436,000","74,420,000","16,976,763","16,976,763"
2019-09-30,"338,516,000","248,028,000","90,488,000","182,295,000","90,488,000","90,488,000","57,101,000","198,535,000","90,488,000","108,047,000","59,203,000","17,772,944","17,772,944"
2018-09-30,"365,725,000","258,578,000","107,147,000","200,882,000","107,147,000","107,147,000","14,473,000","221,630,000","107,147,000","114,483,000","88,570,000","19,019,944","19,019,944"
2017-09-30,"375,319,000","241,272,000","134,047,000","231,254,000","134,047,000","126,032,000","27,831,000","249,727,000","126,032,000","115,680,000","95,391,000","20,504,804","20,504,804"


In [30]:
def get_cash_flow_statement(ticker):

  url_CashFlowStmt = 'https://finance.yahoo.com/quote/%s/cash-flow?p=%s' %(ticker,ticker)
  read_data_CF = ur.urlopen(url_CashFlowStmt).read() 
  raw_CF = BeautifulSoup(read_data_CF,'lxml')

  ls_CF = [] 
  for data_CF in raw_CF.find_all('div'): 
    ls_CF.append(data_CF.string) 

  CleanData_CF = list(filter(None,ls_CF))

  CleanData_CF = CleanData_CF[13:]
  CleanData_CF.remove('Quarterly')
  CleanData_CF.remove('Expand All')
  CleanData_CF.insert(6,'Operating Cash Flow')
  CleanData_CF.insert(12,'Investing Cash Flow')
  CleanData_CF.insert(18,'Financing Cash Flow')
  CleanData_CF.insert(24,'End Cash Position')

  data_CF = list(zip(*[iter(CleanData_CF)]*6))

  cash_flow = pd.DataFrame(data_CF[:-1])
  cash_flow.iloc[0,0] =""
  cash_flow.columns = cash_flow.iloc[0] # Name columns to first row of dataframe
  cash_flow = cash_flow.iloc[1:,] # start to read 1st row
  cash_flow = cash_flow.T # transpose dataframe
  cash_flow.columns = cash_flow.iloc[0] #Name columns to first row of dataframe
  cash_flow.drop(cash_flow.index[0],inplace=True) #Drop first index row
  cash_flow.index.name = '' # Remove the index name
  cash_flow.rename(index={'ttm': '%s/%s/%s' %(date.today().month,date.today().day,date.today().year)},inplace=True) #Rename ttm in index columns to end of the year
  cash_flow.index = pd.to_datetime(cash_flow.index.values,infer_datetime_format=True) # Converting the index as date

  return cash_flow

In [31]:
CF = get_cash_flow_statement(ticker='AAPL')

In [32]:
CF

,Operating Cash Flow,Investing Cash Flow,Financing Cash Flow,End Cash Position,Income Tax Paid Supplemental Data,Interest Paid Supplemental Data,Capital Expenditure,Issuance of Capital Stock,Issuance of Debt,Repayment of Debt,Repurchase of Capital Stock
2020-11-17,"80,008,000","-10,618,000","-86,502,000","35,039,000","11,878,000","3,135,000","-8,302,000","820,000","22,794,000","-13,908,000","-72,615,000"
2020-09-30,"80,674,000","-4,289,000","-86,820,000","39,789,000","9,501,000","3,002,000","-7,309,000","880,000","16,091,000","-12,629,000","-72,358,000"
2019-09-30,"69,391,000","45,896,000","-90,976,000","50,224,000","15,263,000","3,423,000","-10,495,000","781,000","6,963,000","-8,805,000","-66,897,000"
2018-09-30,"77,434,000","16,066,000","-87,876,000","25,913,000","10,417,000","3,022,000","-13,313,000","669,000","6,969,000","-6,500,000","-72,738,000"
2017-09-30,"63,598,000","-46,446,000","-17,347,000","20,289,000","11,591,000","2,092,000","-12,795,000","555,000","28,662,000","-3,500,000","-32,900,000"


### **Remarks:**

It is unfortunate but I didn't found a way to switch from annual to quaterly for the frequence of the data.

That's interesting to do but requires a long analysis of the html that we get from beautiful soup.

Fundamentals data is extremely regularized and low frequency, so, there is not much to be exploted except if we combine them with some other data (ex: stock prices)

One huge drawback that comes with fundamentals data is their modification/correction after released because it will eventually lead to backtesting errors. It would be a good practice to specify the date of released  and last correction  for data vendors.

I recomand to take a look at SimFin that seems to have done a big effort in this way. Their data are free to use until one year prior the current one.